In [1]:
!pip -q install transformers
!pip -q install youtube_dl

     |████████████████████████████████| 2.1MB 18.7MB/s 
     |████████████████████████████████| 3.3MB 45.6MB/s 
     |████████████████████████████████| 901kB 49.2MB/s 
     |████████████████████████████████| 1.9MB 22.7MB/s 


In [2]:
import os
import torch
import librosa

import moviepy.editor as mp

from IPython.display import Audio
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3743744/45929032 bytes (8.2%)7553024/45929032 bytes (16.4%)11657216/45929032 bytes (25.4%)15654912/45929032 bytes (34.1%)19849216/45929032 bytes (43.2%)24051712/45929032 bytes (52.4%)28139520/45929032 bytes (61.3%)32350208/45929032 bytes (70.4%)36446208/45929032 bytes (79.4%)40566784/45929032 bytes (88.3%)44662784/45929032 bytes (97.2%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [3]:
# Substitute below YT link
clip = "https://www.youtube.com/watch?v=Bsoth9BhS9w"

# Substitue below for start/end seconds
start = 1
end = 60

In [4]:
os.system('youtube-dl {} --recode-video mp4'.format(clip))
os.system('mv *.mp4 clip.mp4')

256